# The Basics: Circuits, Qibo and Qiboconnection

## A Bitflip With Qibo

Probably, the most basic operation that we can perform on a qbit is a bitflip. This operation consist on, given any basis state, inverting its value: 
$$ \left| 0 \right> \rightarrow \left| 1 \right>, \left| 1 \right> \rightarrow \left| 0 \right>$$


Given that we always take meassurements on the Z axis, the bitflip can be applied by performing a simple Pauli X-Gate. This is, a $\pi$ radian rotation along the X axis.

<img src="./media/x_gate.png" alt="x_gate" class="bg-primary" width="580" style="display=block; margin-left:auto; margin-right:auto"/>



We will use this as a first exercise with Qibo circuits. Lets first import everything we need.
* [Qibo Circuits](https://qibo.science/qibo/stable/api-reference/qibo.html#circuit-models)
* [Qibo Gates](https://qibo.science/qibo/stable/api-reference/qibo.html#gates)

In [2]:
# Imports

from qibo.gates import X, M
from qibo.models.circuit import Circuit

In [3]:
# Build the circuit. Create a circuit instance and add to it an X gate to the 0th qbit and a Meassurement to all qbits.

circuit = Circuit(nqubits=5)
circuit.add(X(0))
circuit.add(M(0, 1, 2, 3, 4))

MeasurementResult(qubits=(0, 1, 2, 3, 4), nshots=0)

We can check the circuit using the `.summary()` and `.draw()` methods.

In [7]:
# Look at the outputs of summary and draw infomrative methods.

print(circuit.draw())

q0: ─X─M─
q1: ───M─
q2: ───M─
q3: ───M─
q4: ───M─


In [8]:
print(circuit.summary())

Circuit depth = 2
Total number of gates = 2
Number of qubits = 5
Most common gates:
x: 1
measure: 1


We can finally call `execute()` to simmulate the execution of the circuit. We can examine the resulting object to see that the results are the expected ones.

For simple circuits, we expect a the results to be an instance of `qibo.result.CircuitResult`, but more complex circuits could also return a `qibo.result.QuantumState`, or a `qibo.result.MeasurementOutcomes`. Choosing a different qibo backend will also lead to different result objects. Docs are available at:
* [Circuit.execute()](https://qibo.science/qibo/stable/api-reference/qibo.html#qibo.models.circuit.Circuit.execute)
* [CircuitResult](https://qibo.science/qibo/stable/api-reference/qibo.html#qibo.result.CircuitResult)
* [QuantumState](https://qibo.science/qibo/stable/api-reference/qibo.html#qibo.result.QuantumState)
* [MeasurementOutcomes](https://qibo.science/qibo/stable/api-reference/qibo.html#qibo.result.MeasurementOutcomes)

In [15]:
# Execute the circuit.

results = circuit.execute(nshots=1000)
type(results)

qibo.result.CircuitResult

Several methods exist for examining the outcome of an execution. `frequencies()` counts the ocurrences of each state, `probabilities()` computes the `frequencies() / nshots` for each possible state in order, and `samples()` **generates** a distribution of states based on the theoretical distribution. More info on the api for each class, linked above.

In [18]:
# Check the frequencies, probabilities and sample methods.
results.frequencies()

Counter({'10000': 1000})

In [22]:
results.probabilities()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
results.samples()

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=int32)

## Circuits through Qiboconnection

Unless specified otherwise, qibo executions will happen locally in simulation. This is fine for small circuits, but may only lead you so far. We are going to see how we can send executions to Qilimanjaro's Quantum As A Service.

For doing so, first things first, you need to create a `.env` file in the root of this repo, and add there your credentials for our service: 

```env
#.env
API_USER = <insert here your username, without brackets>
API_KEY = <insert here you api-key, without brackets>
```

In [21]:
# Do the imports, and load .env file into env variables available in the python session.

from qiboconnection import API
from dotenv import load_dotenv
import os

load_dotenv()

True

In `qiboconnection`, the central class is `API`. Once instanced with the credentials, it exposes the main methods for iteracting with Qilimanjaro's QGQS.

In [23]:
# Log into QGQS creting an API instance with its .login() constructor.

api = API.login(
    username=os.getenv("API_USER"),
    api_key=os.getenv("API_KEY"),
)

`API.list_devices` allows to see which devices we have available for execution. We can queue jobs despite their current status.

In [24]:
# Use API.list_devices() to see the available devices.

api.list_devices()

<Devices[2]:[
  {
    "status": "online",
    "type": "device",
    "number_pending_jobs": 0,
    "name": "Saruman",
    "id": 16
  },
  {
    "status": "offline",
    "type": "device",
    "number_pending_jobs": 0,
    "name": "Radagast Simulator",
    "id": 1
  }
]>

Then, we can issue the execution of a job by calling the instance method `API.execute()`. For starters, we can execute the same circuit we have above, and compare the results. To execute, we can send a qibo circuit, a list of qibo circuits, or a `qpgrogram` (more info on that later).

In [19]:
# Issue an execution and save the execution id into a variable.

job_id = api.execute(circuit=circuit, device_id=16, nshots=10)
job_id

13277

In [21]:
# Recover the job data by calling API.get_job() with your assigned execution ID. Your execution might spend some time before being executed, after calling get_job(), you can check the jobs queue_position and status.

job = api.get_job(13277)
job.status

Your job with id 13277 is completed.


'completed'

In [22]:
# Once the job status is completed, check its result attribute.

job.result

[{'probabilities': {'00000': 0.1,
   '10000': 0.5,
   '10010': 0.1,
   '10100': 0.1,
   '11000': 0.2},
  'counts': {'00000': 1, '10000': 5, '10010': 1, '10100': 1, '11000': 2},
  'samples': [[1, 1, 0, 0, 0],
   [1, 0, 0, 0, 0],
   [1, 0, 0, 0, 0],
   [1, 1, 0, 0, 0],
   [0, 0, 0, 0, 0],
   [1, 0, 1, 0, 0],
   [1, 0, 0, 0, 0],
   [1, 0, 0, 0, 0],
   [1, 0, 0, 0, 0],
   [1, 0, 0, 1, 0]]}]